# How to Store Simulations to HDF

You can ask TARDIS to store the state of each iteration of the simulation you are running. We show examples of how this is done:

Initialize the simulation with the `tardis_example.yml` configuration file.

In [1]:
from tardis import run_tardis
from tardis.io.atom_data import download_atom_data

# We download the atomic data needed to run the simulation
download_atom_data('kurucz_cd23_chianti_H_He')

# We run the simulation
simulation = run_tardis('tardis_example.yml')

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Atomic Data kurucz_cd23_chianti_H_He already exists in /home/runner/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5. Will not download - override with force_download=True.


Tabs(height=350, sizing_mode='stretch_width')
    [0] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [1] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [2] HTML(str, height=300, styles={'overflow-y': 'auto', ...})
    [3] HTML(str, height=300, styles={'overflow-y': 'auto', ...})

TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:\u2007\u2007\u2007', layout=Layout(width='6%')), FloatProgress(value=0.…

You can now use the `to_hdf` method, to save properties to a HDF file.

#### Parameters  

`file_path`: Path where the HDF file should be stored. (Required)  
`path`: Path inside the HDF store to store the elements. (Optional)  
`name`: Name of the group inside HDF store, under which properties will be saved.(Optional)
`overwrite`: If the HDF file already exists, do you overwrite the existing file (Optional, default `False`)

<div class="alert alert-info">

Note
    
Throughout this notebook, we set ``overwrite=True`` so that the notebook can be run repeatedly if needed.

</div>

In [2]:
simulation.to_hdf('/tmp/full_example.hdf', overwrite=True)

# The commented out code below shows an example of to_hdf with more parameters
#simulation.to_hdf(file_path='/tmp/full_example.hdf', path='/', name='simulation')

Open the stored HDF file with pandas and print a list of its entries using the `keys()` method:

In [3]:
import pandas as pd

data = pd.HDFStore('/tmp/full_example.hdf', overwrite=True)

data.keys()

['/simulation/iterations_electron_densities',
 '/simulation/iterations_t_inner',
 '/simulation/iterations_t_rad',
 '/simulation/iterations_w',
 '/simulation/transport/transport_state/emitted_packet_mask',
 '/simulation/transport/transport_state/j_blue_estimator',
 '/simulation/transport/transport_state/j_estimator',
 '/simulation/transport/transport_state/last_interaction_in_nu',
 '/simulation/transport/transport_state/last_interaction_in_r',
 '/simulation/transport/transport_state/last_interaction_type',
 '/simulation/transport/transport_state/last_line_interaction_in_id',
 '/simulation/transport/transport_state/last_line_interaction_out_id',
 '/simulation/transport/transport_state/last_line_interaction_shell_id',
 '/simulation/transport/transport_state/nu_bar_estimator',
 '/simulation/transport/transport_state/output_energy',
 '/simulation/transport/transport_state/output_nu',
 '/simulation/transport/transport_state/packet_luminosity',
 '/simulation/transport/transport_state/scalars'

Access `model.density` under simulation, which is a one-dimensional array

In [4]:
print(data['/simulation/simulation_state/density'])

0     7.542804e-14
1     5.728475e-14
2     4.396074e-14
3     3.406287e-14
4     2.663135e-14
5     2.099596e-14
6     1.668287e-14
7     1.335311e-14
8     1.076154e-14
9     8.729085e-15
10    7.123652e-15
11    5.846921e-15
12    4.825093e-15
13    4.002324e-15
14    3.336039e-15
15    2.793540e-15
16    2.349550e-15
17    1.984397e-15
18    1.682677e-15
19    1.432260e-15
dtype: float64


Scalars are stored in a `scalars` `pandas.Series` for every module. For example to access `model.t_inner` under simulation, one would need to do the following.

In [5]:
print(data['/simulation/simulation_state/scalars']['t_inner'])

10651.315137263384


## Breakdown of the various to_hdf methods
Every module in TARDIS has its own `to_hdf` method responsible to store its own data to an HDF file.

### Plasma
The following call will store every plasma property to `/tmp/plasma_output.hdf` under `/parent/plasma`

In [6]:
simulation.plasma.to_hdf('/tmp/plasma_output.hdf', path='parent', overwrite=True)

In [7]:
import pandas

plasma_data = pandas.HDFStore('/tmp/plasma_output.hdf')

plasma_data.keys()

['/parent/plasma/beta_rad',
 '/parent/plasma/beta_sobolev',
 '/parent/plasma/continuum_interaction_species',
 '/parent/plasma/dilute_planckian_radiation_field',
 '/parent/plasma/electron_densities',
 '/parent/plasma/excitation_energy',
 '/parent/plasma/f_lu',
 '/parent/plasma/g',
 '/parent/plasma/g_electron',
 '/parent/plasma/general_level_boltzmann_factor',
 '/parent/plasma/ion_number_density',
 '/parent/plasma/ionization_data',
 '/parent/plasma/j_blues',
 '/parent/plasma/level_boltzmann_factor',
 '/parent/plasma/level_number_density',
 '/parent/plasma/levels',
 '/parent/plasma/lines',
 '/parent/plasma/lines_lower_level_index',
 '/parent/plasma/lines_upper_level_index',
 '/parent/plasma/macro_atom_data',
 '/parent/plasma/metastability',
 '/parent/plasma/nlte_excitation_species',
 '/parent/plasma/nlte_ionization_species',
 '/parent/plasma/nu',
 '/parent/plasma/number_density',
 '/parent/plasma/partition_function',
 '/parent/plasma/phi',
 '/parent/plasma/scalars',
 '/parent/plasma/selec

Plasma's `to_hdf` method can also accept a `collection` parameter which can specify which types of plasma properties will be stored. For example if we wanted to only store Input plasma properties, we would do the following:

In [8]:
from tardis.plasma.properties.base import Input
simulation.plasma.to_hdf('/tmp/plasma_input_output.hdf', collection=[Input], overwrite=True)

In [9]:
import pandas

plasma_input_data = pandas.HDFStore('/tmp/plasma_input_output.hdf')

plasma_input_data.keys()

['/plasma/continuum_interaction_species',
 '/plasma/dilute_planckian_radiation_field',
 '/plasma/j_blues',
 '/plasma/nlte_excitation_species',
 '/plasma/nlte_ionization_species',
 '/plasma/number_density',
 '/plasma/scalars']

### Model
The following call will store properties of the `SimulationState` to `/tmp/model_output.hdf` under `/simulation_state`.

In [10]:
simulation.simulation_state.to_hdf('/tmp/model_output.hdf', overwrite=True)

In [11]:
import pandas

model_data = pandas.HDFStore('/tmp/model_output.hdf')

model_data.keys()

['/simulation_state/abundance',
 '/simulation_state/density',
 '/simulation_state/dilution_factor',
 '/simulation_state/r_inner',
 '/simulation_state/scalars',
 '/simulation_state/t_radiative',
 '/simulation_state/v_inner',
 '/simulation_state/v_outer']

### MontecarloTransport
The following call will store properties of the `MontecarloTransport` to `/tmp/transport_output.hdf` under `/transport`.

In [12]:
simulation.transport.to_hdf('/tmp/transport_output.hdf', overwrite=True)

In [13]:
import pandas

transport_data = pandas.HDFStore('/tmp/transport_output.hdf')

transport_data.keys()

['/transport/transport_state/emitted_packet_mask',
 '/transport/transport_state/j_blue_estimator',
 '/transport/transport_state/j_estimator',
 '/transport/transport_state/last_interaction_in_nu',
 '/transport/transport_state/last_interaction_in_r',
 '/transport/transport_state/last_interaction_type',
 '/transport/transport_state/last_line_interaction_in_id',
 '/transport/transport_state/last_line_interaction_out_id',
 '/transport/transport_state/last_line_interaction_shell_id',
 '/transport/transport_state/nu_bar_estimator',
 '/transport/transport_state/output_energy',
 '/transport/transport_state/output_nu',
 '/transport/transport_state/packet_luminosity',
 '/transport/transport_state/scalars']